In [89]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import uuid

In [110]:
data = pd.read_csv('./data/data.csv')
data

,Unnamed: 0,created_at,text,coordinates
0,0,2013-12-31 07:14:22+00:00,Boom bitch get out the way! #drunk #islands #g...,"[-118.10041174, 34.14628356]"
1,1,2013-12-30 20:29:20+00:00,Story of my life! 😂 #boulder http://t.co/ZMfNK...,"[0.13429814, 52.22500698]"
2,2,2014-01-01 06:12:15+00:00,Happy New Year #Boulder !!!! What are some of ...,"[144.98467167, -37.80312131]"
3,3,2013-12-31 22:48:10+00:00,Last sunset here in #boulder #flatirons @ Dush...,"[-105.27725101, 40.01544423]"
4,4,2014-01-01 04:57:01+00:00,Ringing in the #NewYear @BMoCA for their NYE a...,"[-105.2769568, 40.01579474]"
...,...,...,...,...
3938,3938,2013-09-12 05:08:40+00:00,“@dailycamera: RT @JoeCarmenRubino: #Boulder C...,"[-105.0837946, 40.00217227]"
3939,3939,2013-09-12 05:09:24+00:00,Wow. “@jrbates: Update: @dailycamera reports s...,"[-104.95341443, 39.70087895]"
3940,3940,2013-09-12 04:36:22+00:00,Dealing with heavy rain along I-70E near Auror...,"[-104.87204539, 39.77646518]"
3941,3941,2013-09-12 04:35:37+00:00,CU kids having fun in the flooding underpass. ...,"[-105.27772709, 40.03908039]"


In [111]:
# take only date of day
data['created_at'] = data['created_at'].str.split(' ', expand = True)[0]

In [112]:
data

,Unnamed: 0,created_at,text,coordinates
0,0,2013-12-31,Boom bitch get out the way! #drunk #islands #g...,"[-118.10041174, 34.14628356]"
1,1,2013-12-30,Story of my life! 😂 #boulder http://t.co/ZMfNK...,"[0.13429814, 52.22500698]"
2,2,2014-01-01,Happy New Year #Boulder !!!! What are some of ...,"[144.98467167, -37.80312131]"
3,3,2013-12-31,Last sunset here in #boulder #flatirons @ Dush...,"[-105.27725101, 40.01544423]"
4,4,2014-01-01,Ringing in the #NewYear @BMoCA for their NYE a...,"[-105.2769568, 40.01579474]"
...,...,...,...,...
3938,3938,2013-09-12,“@dailycamera: RT @JoeCarmenRubino: #Boulder C...,"[-105.0837946, 40.00217227]"
3939,3939,2013-09-12,Wow. “@jrbates: Update: @dailycamera reports s...,"[-104.95341443, 39.70087895]"
3940,3940,2013-09-12,Dealing with heavy rain along I-70E near Auror...,"[-104.87204539, 39.77646518]"
3941,3941,2013-09-12,CU kids having fun in the flooding underpass. ...,"[-105.27772709, 40.03908039]"


In [113]:
# delete index column
data = data.drop('Unnamed: 0', 1)

C:\Users\mo6tz\AppData\Local\Temp/ipykernel_19948/139094957.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('Unnamed: 0', 1)


In [105]:
# conveart dataFrame to list
data_list = data.values.tolist()

In [106]:
# conveart coordinates from string to list
for i in range(len(data_list)):
    data_list[i][2] =  data_list[i][2].replace('[', '').replace(']', '').split(',')
    data_list[i][2][0] = float(data_list[i][2][0]) 
    data_list[i][2][1] = float(data_list[i][2][1]) 

In [96]:
es = Elasticsearch()
index_name = 'tweets'

In [97]:
# create an index with custom mapping
es.indices.create(index=index_name, ignore=400, body={
    'mappings': {
        'dynamic': 'strict',
        'properties':{
            'created_at': {'type': 'date'},
            'text': {'type': 'text'},
            'coordinates': {'type': 'geo_point'}
        }
    }
})

C:\Users\mo6tz\AppData\Local\Temp/ipykernel_19948/3132206947.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index_name, ignore=400, body={
C:\Python39\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweets'}

In [107]:
actions = [
    {
        '_index': index_name,
        '_type': '_doc',
        '_id': uuid.uuid4().int,
        '_source': {
            "created_at": tweet[0],
            "text": tweet[1],
            "coordinates": tweet[2]
        }
    }
    for tweet in data_list
]


In [108]:
actions[0]

{'_index': 'tweets',
 '_type': '_doc',
 '_id': 264345021700784448705878855496313140130,
 '_source': {'created_at': '2013-12-31',
  'text': 'Boom bitch get out the way! #drunk #islands #girlsnight  #BJs #hookah #zephyrs #boulder #marines… http://t.co/uYmu7c4o0x',
  'coordinates': [-118.10041174, 34.14628356]}}

In [109]:
helpers.bulk(es, actions)

C:\Python39\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
C:\Python39\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


(3943, [])